#Cats vs. Dogs

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn import linear_model
import cPickle as pickle
from time import time
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
import random
from sklearn.decomposition import PCA

##Классификация с использованием гистограмм визуальных слов

In [6]:
#calclulate descriptors of the images generated by image_generator
def get_descriptors(image_generator, images):
    descriptors = []
    sift = cv2.SIFT(nfeatures=100)
    for image, cls in image_generator(images):
        keys, desc = sift.detectAndCompute(image, None)
        for d, k in zip(desc, keys):
            descriptors.append(d)
    return descriptors

def read_results_set(filename):
    y_set = {}
    with open(filename, 'r') as fin:
        for line in fin.readlines():
            image_name, cls = tuple(line.split(','))
            y_set[image_name] = cls
    return y_set

#generator of train images for the first task
def train_image_generator(image_names):
    train_dir = "train_images\\train_images"
    y_set = read_results_set('train_final.csv')
    for image_name in image_names:
        image = cv2.imread(train_dir+"\\"+ image_name)
        cls = y_set[image_name.split('.')[0]]
        yield cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), cls

#construct the visual dictionary
def get_visual_dictionary(clusters_number, image_generator, images):
    descriptors = get_descriptors(image_generator, images)
    cls = KMeans(n_clusters=clusters_number)
    cls.fit(descriptors)
    return cls

train_images_names = os.listdir("train_images\\train_images")
validate_images_names = train_images_names[int(len(train_images_names)*0.9):]
train_image_names = train_images_names[:int(len(train_images_names)*0.9)]

visual_dict_size = 50
visual_dict = get_visual_dictionary(visual_dict_size, train_image_generator, train_images_names)
pickle.dump(visual_dict, open('visual_dict.pkl', 'wb'))
visual_dict = pickle.load(open('visual_dict.pkl', 'rb'))

#generator of the train images for the second task
def test_image_generator():
    train_dir = "test_images\\test_images"
    for image_name in os.listdir(train_dir):
        image = cv2.imread(train_dir+"\\"+ image_name)
        yield cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), image_name

#calculate histogram Bag-of-Words
def get_Bag_of_Words(image, visual_dict, features):
    hist_size = visual_dict.get_params()['n_clusters']
    hist = np.zeros(hist_size)
    sift = cv2.SIFT(nfeatures=features)
    keys, desc = sift.detectAndCompute(image, None)
    for d in desc:
        y = visual_dict.predict(d)
        hist[y-1] = hist[y-1] + 1
    max_val = max(hist)
    return [val / max_val for val in hist]

def get_data_set(image_generator, images, visual_dict):
    X = []
    y =[]
    for image, cls in image_generator(images):
        X.append(get_Bag_of_Words(image, visual_dict, 100))
        y.append(cls)
    return X, y

def get_result_file(X_train, y_train, visual_dict, image_generator):
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    with open("result.csv", 'w') as fout:
        fout.write('Id,Prediction\n')
        for image, name in image_generator():
            y = clf.predict(get_Bag_of_Words(image, visual_dict, 100))
            fout.write(name.split('.')[0] +','+str(y[0]))
            
#solve the second task: train SVM classifier and measure its accuracy
def solve(visual_dict):
    X_train, y_train = get_data_set(train_image_generator, train_image_names, visual_dict)
    X_validate, y_validate = get_data_set(train_image_generator, validate_images_names, visual_dict)
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_validate, y_validate)
    print "Accuracy: {}".format(accuracy)
    return clf
    
clf = solve(visual_dict)

Accuracy: 0.70652173913


In [7]:
X_train, y_train = get_data_set(train_image_generator, train_image_names, visual_dict)
X_validate, y_validate = get_data_set(train_image_generator, validate_images_names, visual_dict)

X_train = X_train + X_validate
y_train = y_train + y_validate
get_result_file(X_train, y_train, visual_dict, test_image_generator)

Результат сабмита:0.54356

##Дескрипторы fc7

In [8]:
#generator of train images for the first task
def get_train_data(image_names):
    train_dir = "train_fc7\\train_fc7"
    y_set = read_results_set('train_final.csv')
    X = []
    y = []
    for image_name in image_names:
        desc_name = image_name.split('.')[0] +'.desc'
        X.append(np.loadtxt(train_dir+"\\"+ desc_name))
        y.append(y_set[image_name.split('.')[0]])
    return X, y

#generator of train images for the first task
def test_data_generator():
    train_dir = "test_fc7\\test_fc7"
    for image_name in os.listdir(train_dir):
        desc_name = image_name.split('.')[0] +'.desc'
        desc = np.loadtxt(train_dir+"\\"+ desc_name)
        yield desc, image_name     

def get_result_file(X_train, y_train, image_generator):
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    with open("fc7_result.csv", 'w') as fout:
        fout.write('Id,Prediction\n')
        for desc, name in image_generator():
            y = clf.predict(desc)
            fout.write(name.split('.')[0] +','+str(y[0]))
            
#solve the second task: train SVM classifier and measure its accuracy
def solve():
    X_train, y_train = get_train_data(train_image_names)
    X_validate, y_validate = get_train_data(validate_images_names)
    clf = svm.SVC()
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_validate, y_validate)
    print "Accuracy: {0:0.3f}".format(accuracy)
    return clf
    
clf = solve()

Accuracy: 0.810


In [5]:
X_train, y_train = get_train_data(train_image_names)
X_validate, y_validate = get_train_data(validate_images_names)
X_train = X_train + X_validate
y_train = y_train + y_validate
get_result_file(X_train, y_train, test_data_generator)

Результат сабмита: 0.6241

##Эксперименты

In [7]:
X_train, y_train = get_train_data(train_image_names)
X_validate, y_validate = get_train_data(validate_images_names)

In [11]:
pca = PCA()
train_pca = [desc[:1500] for desc in pca.fit_transform(X_train)]
test_pca = [desc[:1500] for desc in pca.transform(X_validate)]
clf = svm.SVC()
print 'Training..'
clf.fit(train_pca, y_train)
print 'Classifying..'
accuracy = clf.score(test_pca, y_validate)
print "Accuracy: {0:0.3f}".format(accuracy)

Training..
Classifying..
Accuracy: 0.821


In [15]:
X_train = X_train + X_validate
y_train = y_train + y_validate
def get_result_file(X_train, y_train, image_generator):
    pca = PCA()
    train_pca = [desc[:1500] for desc in pca.fit_transform(X_train)]
    clf = svm.SVC()
    print 'Training..'
    clf.fit(train_pca, y_train)
    print 'Classifying..'
    with open("fc7_result.csv", 'w') as fout:
        fout.write('Id,Prediction\n')
        for desc, name in image_generator():
            desc = pca.transform(desc)[0][:1500]
            y = clf.predict(desc)
            fout.write(name.split('.')[0] +','+str(y[0]))
get_result_file(X_train, y_train, test_data_generator)            

Training..
Classifying..


Результат сабмита: 0.76407